# Multiple Sclerosis (MS) lesion segmentation of MRI images using a cascade of two 3D convolutional neural networks 


This script assumes that `Lasagne` and `nolearn` have been installed correctly and `CUDA / CUDNN` are configured. 

Import libraries: 

In [55]:
%load_ext autoreload
%autoreload 2

import os
from collections import OrderedDict
from base import *
from build_model_nolearn import cascade_model
from config import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Model configuration:
Configure the model options. Options are passed to the model using the dictionary `options`. The main options are:

In [ ]:
options = {}

# --------------------------------------------------
# Experiment parameters
# --------------------------------------------------

# image modalities used (T1, FLAIR, PD, T2, ...) 
options['modalities'] = ['T1', 'FLAIR']

# Select an experiment name to store net weights and segmentation masks
options['experiment'] = 'test_CNN'

# In order to expose the classifier to more challeging samples, a threshold can be used to to select 
# candidate voxels for training. Note that images are internally normalized to 0 mean 1 standard deviation 
# before applying thresholding. So a value of t > 0.5 on FLAIR is reasonable in most cases to extract 
# all WM lesion candidates
options['min_th'] = 0.5

# randomize training features before fitting the model.  
options['randomize_train'] = True

# Select between pixel-wise or fully-convolutional training models. Although implemented, fully-convolutional
# models have been not tested with this cascaded model 
options['fully_convolutional'] = False


# --------------------------------------------------
# model parameters
# --------------------------------------------------

# 3D patch size. So, far only implemented for 3D CNN models. 
options['patch_size'] = (11,11,11)

# percentage of the training vector that is going to be used to validate the model during training
options['train_split'] = 0.25

# maximum number of epochs used to train the model
options['max_epochs'] = 200

# maximum number of epochs without improving validation before stopping training (early stopping) 
options['patience'] = 25

# Number of samples used to test at once. This parameter should be around 50000 for machines
# with less than 32GB of RAM
options['batch_size'] = 50000

# net print verbosity. Set to zero for this particular notebook, but setting this value to 11 is recommended
options['net_verbose'] = 0

# post-processing binary threshold. After segmentation, probabilistic masks are binarized using a defined threshold.
options['t_bin'] = 0.8

# The resulting binary mask is filtered by removing lesion regions with lesion size before a defined value
options['l_min'] = 20


## Experiment configuration:

Organize the experiment. Although not necessary, intermediate results, network weights and final lesion segmentation masks are stored inside a folder with name `options['experiment']`. This is extremely useful when a lot of experiments are computed on the same images to declutter the user space. 

In [ ]:
exp_folder = os.path.join(os.getcwd(), options['experiment'])
if not os.path.exists(exp_folder):
    os.mkdir(exp_folder)
    os.mkdir(os.path.join(exp_folder,'nets'))
    os.mkdir(os.path.join(exp_folder,'.train'))

# set the output name 
options['test_name'] = 'cnn_' + options['experiment'] + '.nii.gz'
 
    

## Load the training data:

Training data is internally loaded by the method. So far, training and testing images are passed as dictionaries, where each training image is stored as follows: 

```
traininig_X_data['image_identifier'] = {'modality_1': /path/to/image_modality_n.nii.gz/,
                                         ....
                                        'modality_n': /path/to/image_modality_n.nii.gz/}
```

And also for labels: 

```
traininig_y_data['image_identifier_1'] = 'path/to/image_labels.nii.gz/'
```

**NOTE**: As stated in the paper, input images have been already skull-stripped and bias corrected (N3, etc...) by the user before running the classifer.

In [11]:
train_folder = '/mnt/DATA/w/CNN/images/train_images'
train_x_data = {}
train_y_data = {}

# TRAIN X DATA
train_x_data['im1'] = {'T1': os.path.join(train_folder,'im1', 'T1.nii.gz'), 
                       'FLAIR': os.path.join(train_folder,'im1', 'T1.nii.gz')}
train_x_data['im2'] = {'T1': os.path.join(train_folder,'im2', 'T1.nii.gz'), 
                       'FLAIR': os.path.join(train_folder,'im2', 'T1.nii.gz')}
train_x_data['im3'] = {'T1': os.path.join(train_folder,'im3', 'T1.nii.gz'), 
                       'FLAIR': os.path.join(train_folder,'im3', 'T1.nii.gz')}

# TRAIN LABELS 
train_y_data['im1'] = os.path.join(train_folder,'im1', 'lesion_bin.nii.gz')
train_y_data['im2'] = os.path.join(train_folder,'im2', 'lesion_bin.nii.gz')
train_y_data['im3'] = os.path.join(train_folder,'im3', 'lesion_bin.nii.gz')



## Initialize the model:

The model is initialized using the function `cascade_model`, which returns a list of two `NeuralNet` objects. Optimized weights are stored also inside the experiment folder for future use (testing different images without re-training the model.  

In [9]:
options['weight_paths'] = os.getcwd()
model = cascade_model(options)


## Train the model:

The function `train_cascaded_model` is used to train the model. The next image summarizes the training procedure. For further information about how this function optimizes the two CNN, please consult the original paper. (**NOTE**: For this example, `options['net_verbose`] has been set to `0` for simplicity)


![](pipeline_training.png)



In [64]:
model = train_cascaded_model(model, train_x_data, train_y_data,  options)


---> cnn1 loading training data
---> cnn1 train_x  (32290, 2, 11, 11, 11) 

Early stopping.
Best valid loss was 0.046777 at epoch 77.
---> cnn2 loading training data
---> cnn2 train_x  (32290, 2, 11, 11, 11) 

Early stopping.
Best valid loss was 0.165393 at epoch 238.


## Test the model:

Once the model has been trained, it can e tested on other images. Please note that the same image modalities have to be used. Testing images are loaded equally to training_data, so a `dictionary` defines the modalities used:

```
test_X_data['image_identifier'] = {'modality_1': /path/to/image_modality_n.nii.gz/,
                                         ....
                                   'modality_n': /path/to/image_modality_n.nii.gz/}
```


In [66]:
# TEST X DATA
test_folder = '/mnt/DATA/w/CNN/images/test_images'
test_x_data = {}
test_x_data['im1'] = {'T1': os.path.join(test_folder,'im1', 'T1.nii.gz'), 
                       'FLAIR': os.path.join(test_folder,'im1', 'FLAIR.nii.gz')}


# set the output_location of the final segmentation. In this particular example, 
# we are training and testing on the same images
options['test_folder'] = test_folder
options['test_scan'] = 'im1'
out_seg = test_cascaded_model(model, test_x_data, options)


 ---> testing the model


Compute different metrics on tested data:

In [69]:
from metrics import *
# load the GT annotation for the tested image 
GT = nib.load(os.path.join(test_folder,'im1', 'lesion_bin.nii.gz')).get_data()
print 'DSC:',  DSC(GT, out_seg)

[ 0.  1.] [ 0.  1.]
DSC: 0.018423191547
